# Bootstrapped Supervised Model Training

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import r2_score
from scipy.stats import spearmanr
# models
from sklearn.ensemble import RandomForestRegressor

# SVR
from sklearn.svm import SVR

from scipy.stats import ttest_ind

import json

# color map
import matplotlib.cm as cm

In [2]:
msa_kcat_df = pd.read_csv("../data/20240812_adk_msa_temp.csv")
esm_df = pd.read_csv("../data/20240201_adk_esm_650M_layer33_updated.csv", index_col=0)
dataset = pd.read_csv('../data/20240815_one_hot_adk_ml_dataset.csv')

In [3]:
esm_kcat_df = pd.merge(dataset[["org_name", "kcat_mean", "Km_mean", "kcat_km"]], esm_df, left_on="org_name", right_on="Organism")
esm_kcat_df["log10_kcat"] = np.log10(esm_kcat_df["kcat_mean"])
esm_kcat_df["log10_km"] = np.log10(esm_kcat_df["Km_mean"])
esm_kcat_df["log10_kcat_km"] = np.log10(esm_kcat_df["kcat_km"])
esm_kcat_df.drop(columns=[ "kcat_mean", "Km_mean", "kcat_km", "Organism"], inplace=True)


esm_kcat_df

,org_name,0,1,2,3,4,5,6,7,8,...,1273,1274,1275,1276,1277,1278,1279,log10_kcat,log10_km,log10_kcat_km
0,azorhizobium_caulinodans,-0.029848,-0.107304,-0.030239,-0.033325,-0.054049,-0.043837,0.123899,-0.171721,0.012940,...,-0.068868,-0.073802,-0.015658,0.102610,-0.196471,-0.010017,0.087720,1.307235,2.358495,4.948740
1,methylocella_silvestris,-0.025397,-0.098017,-0.064225,0.004590,-0.058120,-0.026560,0.146734,-0.205433,0.017737,...,-0.037979,-0.078104,-0.023356,0.089246,-0.219145,-0.028109,0.099179,2.090121,3.028001,5.062119
2,bartonella_henselae,0.029439,-0.120885,-0.011720,-0.010969,-0.081942,-0.063146,0.103362,-0.140911,-0.000716,...,-0.041773,-0.027291,-0.024682,0.085903,-0.137811,-0.039534,0.093870,1.891296,2.534387,5.356910
3,sorangium_cellulosum,-0.015960,-0.079255,-0.024043,-0.059969,-0.035065,-0.055907,0.080270,-0.066205,0.010298,...,-0.001880,-0.034523,-0.037967,0.075378,-0.148082,0.007141,0.044445,1.097551,2.335052,4.762500
4,elusimicrobium_minutum,0.017232,-0.112872,0.011568,0.005158,-0.065571,-0.032126,0.066255,-0.119295,0.009024,...,0.025108,-0.040576,-0.031321,0.145570,-0.108281,-0.059786,-0.003229,2.141025,3.236850,4.904175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,treponema_putidum,0.008175,-0.138393,-0.022943,0.027759,-0.078267,-0.023235,0.080516,-0.121521,-0.027342,...,0.038950,-0.098080,-0.027557,0.139332,-0.141455,-0.052172,0.029993,2.455905,3.196944,5.258961
171,veillonella_seminalis,-0.011368,-0.103895,-0.044693,-0.003996,-0.111342,-0.029191,0.112389,-0.176406,-0.036035,...,-0.071626,-0.084354,-0.047088,0.067589,-0.210914,0.020568,0.089854,2.121395,2.536187,5.585208
172,verminephrobacter_aporrectodeae,-0.023410,-0.079967,-0.007041,-0.016315,-0.066047,-0.029182,0.102054,-0.108076,0.000317,...,-0.013728,-0.079685,-0.054644,0.038545,-0.201637,0.048119,0.069546,0.157433,2.969472,3.187960
173,virgibacillus_dokdonensis,-0.007836,-0.127056,-0.048668,0.018729,-0.076071,-0.008343,0.142827,-0.178651,-0.040615,...,-0.025948,-0.112436,-0.023977,0.091477,-0.196443,-0.009191,0.070859,2.290008,2.578797,5.711212


In [4]:
dataset = dataset.merge(msa_kcat_df[["org_name", "lid_type"]], left_on="org_name", right_on='org_name', how="left")
# drop the organism_name column
#dataset.drop("organism_name", axis=1, inplace=True)
dataset['log10_kcat'] = np.log10(dataset['kcat_mean'])
dataset['log10_km'] = np.log10(dataset['Km_mean'])
dataset['log10_kcatkm'] = np.log10(dataset['kcat_km'])


In [5]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,7264,7265,org_name,kcat_mean,Km_mean,kcat_km,lid_type,log10_kcat,log10_km,log10_kcatkm
0,0,0,0,0,0,0,0,0,0,0,...,0,1,azorhizobium_caulinodans,20.287820,228.294494,88866.882700,lidless,1.307235,2.358495,4.948740
1,0,0,0,0,0,0,0,0,0,0,...,0,1,methylocella_silvestris,123.061061,1066.598980,115377.065900,lidless,2.090121,3.028001,5.062119
2,0,0,0,0,0,0,0,0,0,0,...,0,1,bartonella_henselae,77.856752,342.284019,227462.421200,lidless,1.891296,2.534387,5.356910
3,0,0,0,0,0,0,0,0,0,0,...,0,1,sorangium_cellulosum,12.518474,216.297684,57876.134080,lidless,1.097551,2.335052,4.762500
4,0,0,0,0,0,0,0,0,0,0,...,0,1,elusimicrobium_minutum,138.364545,1725.243090,80200.028740,zinc-like,2.141025,3.236850,4.904175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,0,0,0,0,0,0,0,0,0,0,...,0,1,treponema_putidum,285.696479,1573.778450,181535.386100,zinc-like,2.455905,3.196944,5.258961
171,0,0,0,0,0,0,0,0,0,0,...,0,1,veillonella_seminalis,132.249815,343.705960,384776.030100,lidless,2.121395,2.536187,5.585208
172,0,0,0,0,0,0,0,0,0,0,...,0,1,verminephrobacter_aporrectodeae,1.436920,932.121022,1541.559318,hbond_like,0.157433,2.969472,3.187960
173,0,0,0,0,0,0,0,0,0,0,...,0,1,virgibacillus_dokdonensis,194.988192,379.137519,514294.106600,zinc-like,2.290008,2.578797,5.711212


In [22]:
# INCREMENT = 20
# N_SAMPLES = 30

# split_dict = {}

# train_size = 0.8 * dataset.shape[0]
# num_sizes = int(train_size//INCREMENT if train_size % INCREMENT == 0 else train_size//INCREMENT)

# assert dataset.shape[0] == 175

# for i in range(N_SAMPLES):
#     np.random.seed(i)

#     train, test = train_test_split(dataset, test_size=0.2, random_state=i, stratify=dataset['lid_type'])
#     d = {"test": test["org_name"].to_list()}
    
#     X_train = train.iloc[:, :-8].values   

#     data_set_sizes = np.arange(INCREMENT,X_train.shape[0], INCREMENT)
#     data_set_sizes = np.append(data_set_sizes, X_train.shape[0])
        

#     assert len(train["org_name"]) == len(set(train["org_name"]))
#     assert len(test["org_name"]) == len(set(test["org_name"]))
#     for j, size in enumerate(data_set_sizes):

#         if size == X_train.shape[0]:
#             train_subset = np.arange(X_train.shape[0])
#         else:
#             train_subset, _ = train_test_split(np.arange(X_train.shape[0]), train_size=size, random_state=i, stratify=train['lid_type'])

#         d["train_size_{}".format(size)] = train["org_name"].values[train_subset].tolist()

#     split_dict[i] = d     

# # write split_dict to json
# with open("../data/20240717_bootstrap_stratified_split.json", "w") as f:
#     json.dump(split_dict, f)

In [6]:
split_dict = json.load(open("../data/20240717_bootstrap_stratified_split.json"))

In [ ]:
INCREMENT = 20
N_SAMPLES = 30

train_size = 0.8 * dataset.shape[0]
num_sizes = int(train_size//INCREMENT if train_size % INCREMENT == 0 else train_size//INCREMENT)

kcat_score_dict = {
    "RF_one-hot": np.zeros((N_SAMPLES, num_sizes)),
    "RF_esm": np.zeros((N_SAMPLES, num_sizes)),
    "SVR_one-hot": np.zeros((N_SAMPLES, num_sizes)),
    "SVR_esm": np.zeros((N_SAMPLES, num_sizes))
}


km_score_dict = {
    "RF_one-hot": np.zeros((N_SAMPLES, num_sizes)),
    "RF_esm": np.zeros((N_SAMPLES, num_sizes)),
    "SVR_one-hot": np.zeros((N_SAMPLES, num_sizes)),
    "SVR_esm": np.zeros((N_SAMPLES, num_sizes))
}


for i in tqdm(list(range(N_SAMPLES))):
    np.random.seed(i)

    train, test = train_test_split(dataset, test_size=0.2, random_state=i, stratify=dataset['lid_type'])
  
    esm_train = esm_kcat_df.loc[train.index]
    esm_test = esm_kcat_df.loc[test.index]

    X_train = train.iloc[:, :-8].values
    y_train_kcat = train['log10_kcat'].values
    y_train_km = train['log10_km'].values

    X_test = test.iloc[:, :-8].values
    y_test_kcat = test['log10_kcat'].values
    y_test_km = test['log10_km'].values

    X_train_esm = esm_train.iloc[:, 1:-3].values
    y_train_kcat_esm = esm_train['log10_kcat'].values
    y_train_km_esm = esm_train['log10_km'].values

    X_test_esm = esm_test.iloc[:, 1:-3].values
    y_test_kcat_esm = esm_test['log10_kcat'].values
    y_test_km_esm = esm_test['log10_km'].values

    data_set_sizes = np.arange(INCREMENT,X_train.shape[0], INCREMENT)
    data_set_sizes = np.append(data_set_sizes, X_train.shape[0])
    if X_train.shape[0] % INCREMENT != 0:
        data_set_sizes = np.append(data_set_sizes, X_train.shape[0])
   

    for j, size in enumerate(data_set_sizes):

        #train_subset = np.random.choice(X_train.shape[0], size, replace=False)
        if size == X_train.shape[0]:
            train_subset = np.arange(X_train.shape[0])
        else:
            train_subset, _ = train_test_split(np.arange(X_train.shape[0]), train_size=size, random_state=i, stratify=train['lid_type'])        
        
        train_subset_orgs = train.values[train_subset][:,-3]
       
        ############################
        # Random Forest
        ############################

        rf = RandomForestRegressor(random_state=314)
        rf.fit(X_train[train_subset], y_train_kcat[train_subset])
        y_pred = rf.predict(X_test)
        # spearman correlation
        kcat_score_dict["RF_one-hot"][i,j] = spearmanr(y_test_kcat, y_pred)[0]

        rf = RandomForestRegressor(random_state=314)
        rf.fit(X_train[train_subset], y_train_km[train_subset])
        y_pred = rf.predict(X_test)
        # spearman correlation
        km_score_dict["RF_one-hot"][i,j] = spearmanr(y_test_km, y_pred)[0]

        rf = RandomForestRegressor(random_state=314)
        rf.fit(X_train_esm[train_subset], y_train_kcat_esm[train_subset])
        y_pred = rf.predict(X_test_esm)
        # spearman correlation
        kcat_score_dict["RF_esm"][i,j] = spearmanr(y_test_kcat_esm, y_pred)[0]

        rf = RandomForestRegressor(random_state=314)
        rf.fit(X_train_esm[train_subset], y_train_km_esm[train_subset])
        y_pred = rf.predict(X_test_esm)
        # spearman correlation
        km_score_dict["RF_esm"][i,j] = spearmanr(y_test_km_esm, y_pred)[0]

        ############################
        # SVR
        ############################

        svr = SVR()
        svr.fit(X_train[train_subset], y_train_kcat[train_subset])
        y_pred = svr.predict(X_test)
        # spearman correlation
        kcat_score_dict["SVR_one-hot"][i,j] = spearmanr(y_test_kcat, y_pred)[0]  

        svr = SVR()
        svr.fit(X_train[train_subset], y_train_km[train_subset])
        y_pred = svr.predict(X_test)
        # spearman correlation
        km_score_dict["SVR_one-hot"][i,j] = spearmanr(y_test_km, y_pred)[0]

        svr = SVR()
        svr.fit(X_train_esm[train_subset], y_train_kcat_esm[train_subset])
        y_pred = svr.predict(X_test_esm)
        # spearman correlation
        kcat_score_dict["SVR_esm"][i,j] = spearmanr(y_test_kcat_esm, y_pred)[0]

        svr = SVR()
        svr.fit(X_train_esm[train_subset], y_train_km_esm[train_subset])
        y_pred = svr.predict(X_test_esm)
        # spearman correlation
        km_score_dict["SVR_esm"][i,j] = spearmanr(y_test_km_esm, y_pred)[0]

       
    

100%|██████████| 30/30 [18:27<00:00, 36.91s/it]


In [10]:
jsonified_kcat_score_dict = {k: v.tolist() for k,v in kcat_score_dict.items()}
jsonified_km_score_dict = {k: v.tolist() for k,v in km_score_dict.items()}

In [12]:
with (open("../data/20241022_score_dict_kcat.json", "w")) as f:
    json.dump(jsonified_kcat_score_dict, f)
with (open("../data/20241022_score_dict_km.json", "w")) as f:
    json.dump(jsonified_km_score_dict, f)